In [1]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import gc
from pathlib import Path
import glob

In [2]:
# ! conda install -c conda-forge python-annoy
from annoy import AnnoyIndex

In [3]:
import hashlib
import os
import gensim
from gensim.models import word2vec
from gensim.models import KeyedVectors
os.environ["PYTHONHASHSEED"] = str(42)
def hashfxn(x):
    return int(hashlib.md5(str(x).encode()).hexdigest(), 16)

In [4]:
type_labels = {'clicks':0, 'carts':1, 'orders':2}
SET = 2

In [5]:
def load_df(inputPath):    
    dfs = []
    for e, chunk_file in enumerate(glob.glob(inputPath)):
        chunk = pd.read_parquet(chunk_file)
        chunk.ts = (chunk.ts/1000).astype('int32')
        chunk['type'] = chunk['type'].map(type_labels).astype('int8')
        dfs.append(chunk)
    return pd.concat(dfs).reset_index(drop=True) #.astype({"ts": "datetime64[ms]"})

In [6]:
if SET == 2:
    train_df = pd.read_parquet('../input/splited/train.parquet')
    valA_df = pd.read_parquet('../input/splited/test.parquet')
if SET == 3:
    train_df = load_df('../input/parquets/train_parquets/*')
    valA_df = load_df('../input/parquets/test_parquets/*')

In [8]:
# def splitByTime(x, days):
#     cnt = 0
#     now = x.iloc[0]
#     distance = days*24*60*60
#     times = []
#     for time in x:
#         if time > now + distance:
#             now = time
#             cnt += 1
#         times.append(cnt)
#     return times


In [9]:
# tqdm.pandas()
# train_df['timeSplit'] = train_df.groupby('session').ts.progress_apply(lambda x: splitByTime(x, 1)).explode('ts').reset_index(drop=True)
# train_df

In [10]:
# tqdm.pandas()
# valA_df['timeSplit'] = valA_df.groupby('session').ts.progress_apply(lambda x: splitByTime(x, 1)).explode('ts').reset_index(drop=True)
# valA_df

In [11]:
# raw_corpus = []
# for session, group_df in tqdm(train_df.groupby(['session', 'timeSplit'])):
#     raw_corpus.append(list(group_df['aid'].astype(str)))
# for session, group_df in tqdm(valA_df.groupby(['session', 'timeSplit'])):
#     raw_corpus.append(list(group_df['aid'].astype(str)))
# raw_corpus

In [7]:
raw_corpus = []
for session, group_df in tqdm(train_df.groupby(['session'])):
    raw_corpus.append(list(group_df['aid'].astype(str)))
for session, group_df in tqdm(valA_df.groupby(['session'])):
    raw_corpus.append(list(group_df['aid'].astype(str)))

A Jupyter Widget

/opt/conda/envs/rapids/lib/python3.9/site-packages/tqdm/std.py:1195: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for obj in iterable:


A Jupyter Widget

/opt/conda/envs/rapids/lib/python3.9/site-packages/tqdm/std.py:1195: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for obj in iterable:


## Aid2Vec

In [13]:
# aid2vec_model = word2vec.Word2Vec(sentences=raw_corpus, vector_size=5, window=5, sg = 0, workers=-1, seed=42, hashfxn=hashfxn)

In [8]:
aid2vec_model = word2vec.Word2Vec(sentences=raw_corpus, vector_size=8, window=3, epochs = 5, sg = 1, ns_exponent=1, min_count=1, workers=4, seed=42, hashfxn=hashfxn)

In [9]:
! mkdir ../output/newSplited/savedModel/set_{SET}/
aid2vec_model.wv.save_word2vec_format(f'../output/newSplited/savedModel/set_{SET}/otto_aid2vec_5d.bin', binary=True)

mkdir: cannot create directory ‘../output/newSplited/savedModel/set_2/’: File exists


In [15]:
aid2vec_model = gensim.models.KeyedVectors.load_word2vec_format('../output/newSplited/savedModel/otto_aid2vec2.bin', binary=True)

In [24]:
aid2vec_model['1517085']

array([ 0.00655328, -0.01009453,  0.0004542 ,  0.00285264, -0.00163658,
        0.0020785 , -0.00442526,  0.00273418,  0.00535741, -0.01939648,
       -0.00214661, -0.00701865, -0.00064074,  0.00690907, -0.01243271,
        0.00276293,  0.00425462, -0.01560698, -0.00198002, -0.00940327,
       -0.00135242, -0.01633408, -0.00964315, -0.01769499, -0.00744589,
       -0.00670934, -0.00286099, -0.01531506,  0.00835216, -0.00640653,
       -0.00234938, -0.01739679,  0.00519398, -0.01843862,  0.00609244,
       -0.00169513,  0.0198422 , -0.0035924 , -0.00485849,  0.006269  ,
        0.01657705, -0.01740865, -0.00161034, -0.01470069, -0.01615997,
        0.0080057 , -0.00400241, -0.01479583,  0.00466196, -0.01018123],
      dtype=float32)

In [ ]:
candidates = aid2vec_model.wv.most_similar(positive=['1517085'], topn=100) 
candidates

## Annoy

In [17]:
aid2idx = {aid: i for i, aid in enumerate(aid2vec_model.index_to_key)}

In [28]:
index = AnnoyIndex(50, 'euclidean')
for aid, idx in aid2idx.items():
    index.add_item(idx, aid2vec_model.vectors[idx])
index.build(10)

True

In [37]:
valA_df

,session,aid,ts,type
0,11098528,11830,1661119200,0
1,11098529,1105029,1661119200,0
2,11098530,264500,1661119200,0
3,11098530,264500,1661119288,0
4,11098530,409236,1661119369,0
...,...,...,...,...
7683572,12899774,33035,1661723968,0
7683573,12899775,1743151,1661723970,0
7683574,12899776,548599,1661723972,0
7683575,12899777,384045,1661723976,0


In [36]:
tqdm.pandas()
valA_df = valA_df.groupby('session').aid.progress_apply(lambda x: list(x.astype(str))).to_frame().reset_index()
valA_df.columns = ["session", "aids"]
valA_df

A Jupyter Widget

,session,aids
0,11098528,[11830]
1,11098529,[1105029]
2,11098530,"[264500, 264500, 409236, 409236, 409236, 409236]"
3,11098531,"[452188, 1239060, 1557766, 452188, 396199, 130..."
4,11098532,"[7651, 876469]"
...,...,...
1801246,12899774,[33035]
1801247,12899775,[1743151]
1801248,12899776,[548599]
1801249,12899777,[384045]


In [54]:
def findSimilarAnnoy(x):
    return [int(aid2vec_model.index_to_key[i]) for i in index.get_nns_by_item(aid2idx[x[-1]], 201)[1:]]

In [58]:
from collections import Counter
def findSimilarAnnoy(x):
    candidates = Counter()
    for aid in x:
        for i in index.get_nns_by_item(aid2idx[aid], 21)[1:]:
            candidates[int(aid2vec_model.index_to_key[i])] += 1
    return [aid for aid, cnt in candidates.most_common(20)]

In [59]:
test = valA_df.iloc[:1000]
test["cand"] = test.aids.progress_apply(lambda x: findSimilarAnnoy(x))
test

A Jupyter Widget

/tmp/ipykernel_91694/2291137037.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["cand"] = test.aids.progress_apply(lambda x: findSimilarAnnoy(x))


,session,aids,cand
0,11098528,[11830],"[1667848, 1813069, 1231718, 126977, 1163731, 1..."
1,11098529,[1105029],"[378499, 277608, 564642, 1069543, 8278, 139796..."
2,11098530,"[264500, 264500, 409236, 409236, 409236, 409236]","[1803337, 511646, 122029, 168229, 1318118, 600..."
3,11098531,"[452188, 1239060, 1557766, 452188, 396199, 130...","[1392409, 281017, 881693, 1759395, 1745374, 14..."
4,11098532,"[7651, 876469]","[373203, 1221708, 1519864, 1241032, 1575836, 2..."
...,...,...,...
995,11099523,"[457885, 1368585, 1738074, 1368585, 453703, 45...","[649379, 1074027, 341414, 215031, 355829, 1046..."
996,11099524,"[216460, 350578]","[1174733, 1550481, 323626, 746370, 706728, 900..."
997,11099525,[46444],"[605096, 1063345, 1037593, 514229, 1557841, 30..."
998,11099526,[485031],"[1301994, 1523836, 1438275, 1192862, 642499, 5..."


In [67]:
valA_df["cand"] = valA_df.aids.progress_apply(lambda x: findSimilarAnnoy(x))
valA_df

A Jupyter Widget

,session,aids,cand
0,11098528,[11830],"[1667848, 1813069, 1231718, 126977, 1163731, 1..."
1,11098529,[1105029],"[378499, 277608, 564642, 1069543, 8278, 139796..."
2,11098530,"[264500, 264500, 409236, 409236, 409236, 409236]","[1803337, 511646, 122029, 168229, 1318118, 600..."
3,11098531,"[452188, 1239060, 1557766, 452188, 396199, 130...","[1392409, 281017, 881693, 1759395, 1745374, 14..."
4,11098532,"[7651, 876469]","[373203, 1221708, 1519864, 1241032, 1575836, 2..."
...,...,...,...
1801246,12899774,[33035],"[1749849, 951029, 1601922, 368555, 916306, 135..."
1801247,12899775,[1743151],"[1847191, 1822834, 167561, 981141, 1732232, 34..."
1801248,12899776,[548599],"[1478889, 802253, 825110, 1086589, 33187, 7296..."
1801249,12899777,[384045],"[824381, 835626, 1534660, 305781, 1291669, 373..."


In [68]:
clicks_pred_df = valA_df[["session", "cand"]]
clicks_pred_df["type"] = 'clicks'
clicks_pred_df.columns = ["session", "labels", "type"]
clicks_pred_df

,session,labels,type
0,11098528,"[1667848, 1813069, 1231718, 126977, 1163731, 1...",clicks
1,11098529,"[378499, 277608, 564642, 1069543, 8278, 139796...",clicks
2,11098530,"[1803337, 511646, 122029, 168229, 1318118, 600...",clicks
3,11098531,"[1392409, 281017, 881693, 1759395, 1745374, 14...",clicks
4,11098532,"[373203, 1221708, 1519864, 1241032, 1575836, 2...",clicks
...,...,...,...
1801246,12899774,"[1749849, 951029, 1601922, 368555, 916306, 135...",clicks
1801247,12899775,"[1847191, 1822834, 167561, 981141, 1732232, 34...",clicks
1801248,12899776,"[1478889, 802253, 825110, 1086589, 33187, 7296...",clicks
1801249,12899777,"[824381, 835626, 1534660, 305781, 1291669, 373...",clicks


In [69]:
carts_pred_df = clicks_pred_df.copy()
carts_pred_df["type"] = 'carts'
carts_pred_df

,session,labels,type
0,11098528,"[1667848, 1813069, 1231718, 126977, 1163731, 1...",carts
1,11098529,"[378499, 277608, 564642, 1069543, 8278, 139796...",carts
2,11098530,"[1803337, 511646, 122029, 168229, 1318118, 600...",carts
3,11098531,"[1392409, 281017, 881693, 1759395, 1745374, 14...",carts
4,11098532,"[373203, 1221708, 1519864, 1241032, 1575836, 2...",carts
...,...,...,...
1801246,12899774,"[1749849, 951029, 1601922, 368555, 916306, 135...",carts
1801247,12899775,"[1847191, 1822834, 167561, 981141, 1732232, 34...",carts
1801248,12899776,"[1478889, 802253, 825110, 1086589, 33187, 7296...",carts
1801249,12899777,"[824381, 835626, 1534660, 305781, 1291669, 373...",carts


In [70]:
buys_pred_df = clicks_pred_df.copy()
buys_pred_df["type"] = 'orders'
buys_pred_df

,session,labels,type
0,11098528,"[1667848, 1813069, 1231718, 126977, 1163731, 1...",orders
1,11098529,"[378499, 277608, 564642, 1069543, 8278, 139796...",orders
2,11098530,"[1803337, 511646, 122029, 168229, 1318118, 600...",orders
3,11098531,"[1392409, 281017, 881693, 1759395, 1745374, 14...",orders
4,11098532,"[373203, 1221708, 1519864, 1241032, 1575836, 2...",orders
...,...,...,...
1801246,12899774,"[1749849, 951029, 1601922, 368555, 916306, 135...",orders
1801247,12899775,"[1847191, 1822834, 167561, 981141, 1732232, 34...",orders
1801248,12899776,"[1478889, 802253, 825110, 1086589, 33187, 7296...",orders
1801249,12899777,"[824381, 835626, 1534660, 305781, 1291669, 373...",orders


In [71]:
pred_df = pd.concat([clicks_pred_df, carts_pred_df, buys_pred_df]).reset_index(drop=True)
pred_df

,session,labels,type
0,11098528,"[1667848, 1813069, 1231718, 126977, 1163731, 1...",clicks
1,11098529,"[378499, 277608, 564642, 1069543, 8278, 139796...",clicks
2,11098530,"[1803337, 511646, 122029, 168229, 1318118, 600...",clicks
3,11098531,"[1392409, 281017, 881693, 1759395, 1745374, 14...",clicks
4,11098532,"[373203, 1221708, 1519864, 1241032, 1575836, 2...",clicks
...,...,...,...
5403748,12899774,"[1749849, 951029, 1601922, 368555, 916306, 135...",orders
5403749,12899775,"[1847191, 1822834, 167561, 981141, 1732232, 34...",orders
5403750,12899776,"[1478889, 802253, 825110, 1086589, 33187, 7296...",orders
5403751,12899777,"[824381, 835626, 1534660, 305781, 1291669, 373...",orders


In [72]:
test_label_path = '../input/splited/test_labels.parquet'
test_labels = pd.read_parquet(test_label_path)
test_labels

,session,type,ground_truth
0,11098528,clicks,[1679529]
1,11098528,carts,[1199737]
2,11098528,orders,"[990658, 950341, 1462506, 1561739, 907564, 369..."
3,11098529,clicks,[1105029]
4,11098530,orders,[409236]
...,...,...,...
2212687,12899774,clicks,[1399483]
2212688,12899775,clicks,[1760714]
2212689,12899776,clicks,[1737908]
2212690,12899777,clicks,[384045]


In [73]:
test_labels = test_labels.merge(pred_df, how='left', on=['session', 'type'])
test_labels['hits'] = test_labels.apply(lambda df: len(set(df.ground_truth).intersection(set(df.labels))), axis=1)
test_labels['gt_count'] = test_labels.ground_truth.str.len().clip(0,20)
test_labels

,session,type,ground_truth,labels,hits,gt_count
0,11098528,clicks,[1679529],"[1667848, 1813069, 1231718, 126977, 1163731, 1...",0,1
1,11098528,carts,[1199737],"[1667848, 1813069, 1231718, 126977, 1163731, 1...",0,1
2,11098528,orders,"[990658, 950341, 1462506, 1561739, 907564, 369...","[1667848, 1813069, 1231718, 126977, 1163731, 1...",0,11
3,11098529,clicks,[1105029],"[378499, 277608, 564642, 1069543, 8278, 139796...",0,1
4,11098530,orders,[409236],"[1803337, 511646, 122029, 168229, 1318118, 600...",0,1
...,...,...,...,...,...,...
2212687,12899774,clicks,[1399483],"[1749849, 951029, 1601922, 368555, 916306, 135...",0,1
2212688,12899775,clicks,[1760714],"[1847191, 1822834, 167561, 981141, 1732232, 34...",0,1
2212689,12899776,clicks,[1737908],"[1478889, 802253, 825110, 1086589, 33187, 7296...",0,1
2212690,12899777,clicks,[384045],"[824381, 835626, 1534660, 305781, 1291669, 373...",0,1


In [74]:
recall_per_type = test_labels.groupby(['type'])['hits'].sum() / test_labels.groupby(['type'])['gt_count'].sum()
print(recall_per_type)
score = (recall_per_type * pd.Series({'clicks': 0.10, 'carts': 0.30, 'orders': 0.60})).sum()
print(score)

type
carts     0.000012
clicks    0.000007
orders    0.000010
dtype: float64
1.0128537491005344e-05
